In [ ]:
import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
# see how MNIST data look like
print("train, valid, test num examples")
print(mnist.train.num_examples, mnist.validation.num_examples, mnist.test.num_examples)

# scikit-image: python imaging library
import skimage.io as io
# how next_batch work?
# you don't need to know at this time...
im = mnist.train.next_batch(1)[0].reshape(28, 28)
io.imshow(im)

## Note
MNIST image is (784,) shape, so if you want plot this you must reshape into (28, 28)

In [ ]:
# set parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf placeholder
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# initializing the variables
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()

sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                      y: batch_ys})
        # compute average loss
        avg_cost += c / total_batch
    # display logs per epoch step
    if (epoch+1) % display_step == 0:
        print("Epoch: {0:04d} cost={1:.9f}" .format(epoch+1, avg_cost))

print("Optimization Finished!")

# test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# calculate accuracy for 3000 examples
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: {0}"
      .format(sess.run(accuracy, feed_dict={x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})))

In [ ]:
inference = tf.argmax(pred, 1)

# plot one example
ex_im = mnist.test.images[1000].reshape(1, -1)
io.imshow(ex_im.reshape(28, 28))
ex_pred = sess.run(inference, feed_dict={x:ex_im})
print(ex_pred)

In [ ]:
sess.close()